In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("../scripts/")
from data_cleaner import DataCleaner
from util import Util

ImportError: cannot import name 'Str1' from 'builtins' (unknown location)

In [3]:
util = Util()
cleaner = DataCleaner()

In [4]:
repo="https://github.com/Amdework21/LLMs-Prompt-Engineering.git"
cleaned_df=util.read_from_dvc("data/news.csv",repo,"news-v1",low_memory=False)

# cleaned_df=util.read_from_file('../data/news.csv')

### our first try structure for co:here API

Task: Score news relevance

Domain: {Domain}

Title: {Title}

Description: {
    long text Description
}

Body: {
    long text Body
}

Analyst_Average_Score : {Analyst_Average_Score}

In [6]:
api_key="fRfFyb4L2dMSdQjRyWspNnf82PjQUQSaAI39cY02"
cleaned_df=cleaner.clean_stopwords(cleaned_df,['Body','Title','Description'])
cleaned_df.head()

,Domain,Title,Description,Body,Link,timestamp,Analyst_Rank,Reference_Final_Score,Analyst_Average_Score
0,rassegnastampa.news,Boris Johnson taxpayer funded jet election cam...,trigger protest vote upset ministers breach Mi...,Boris Johnson taxpayer funded jet election cam...,https://rassegnastampa.news/boris-johnson-usin...,2021-09-09 18:17:46.258006,4,1.96,0.0
1,twitter.com,Stumbled interesting case woman facing evictio...,Stumbled interesting case woman facing evictio...,Stumbled interesting case woman facing evictio...,http://twitter.com/CoruscaKhaya/status/1435585...,2021-09-08 13:02:45.802298,4,12.00,0.0
2,atpe-tchad.info,Marché Résines dans les peintures et revêtemen...,COVID 19 COVID COVID COVID 19 et Post COVID CO...,Le rapport d étude marché Résines dans les pei...,http://atpe-tchad.info/2021/09/13/marche-resin...,2021-09-13 07:32:46.244403,4,0.05,0.0
3,badbluetech.bitnamiapp.com,AI drives data analytics surge study finds,hate raiders linked automated harassment campa...,How drive funnel content marketing link buildi...,http://badbluetech.bitnamiapp.com/p.php?sid=21...,2021-09-11 00:17:45.962605,4,6.10,0.0
4,kryptogazette.com,Triacetin Vertrieb Markt 2021 Globale Unterneh...,Abschnitten und Endanwendungen Organisationen ...,Global Triacetin Vertrieb Markt 2021 von Herst...,https://kryptogazette.com/2021/09/08/triacetin...,2021-09-08 12:47:46.078369,4,0.13,0.0


In [7]:
import cohere
def predict(train_df:pd.DataFrame,test:pd.DataFrame):

    test_df=test.drop(train_df.columns[len(train_df.columns)-1],axis=1)

    prompt=""
    for ind in train_df.index:
        for col in train_df.columns:
            prompt += f"{col}: {train_df.loc[ind,col]}\n\n"
        prompt += "-- --\n\n"

    for col in test_df.columns:
        ind=test_df.index[0]
        prompt += f"{col}: {test_df.loc[ind,col]}\n\n"

    prompt += f"{train_df.columns[len(train_df.columns)-1]}: "
    prompt = "Task: Score News relevance\n\n"+prompt
    
    
    co = cohere.Client(f'{api_key}')
    response = co.generate(
    model='xlarge',
    prompt=prompt,
    max_tokens=4,
    temperature=0.9,
    k=0,
    p=0.75,
    frequency_penalty=0,
    presence_penalty=0,
    stop_sequences=["-- --"],
    return_likelihoods='NONE')
    print('Prediction: {}'.format(response.generations[0].text))
    display(test)

    print("Trained By")
    display(train_df)

    # print(prompt)

In [8]:
all_columns=['Domain', 'Title', 'Description', 'Body', 'Link', 'timestamp','Analyst_Rank', 'Reference_Final_Score', 'Analyst_Average_Score']
useful_columns=['Domain', 'Title', 'Description', 'Body','Analyst_Rank','Analyst_Average_Score']
drop_columns=['Link','Reference_Final_Score','timestamp']

all_train=cleaned_df.head(9)[useful_columns]
unique_train=all_train.drop_duplicates(subset=["Analyst_Average_Score"], keep='first')
unique_train=unique_train.sample(2)
test=cleaned_df.tail(1).drop(drop_columns,axis=1)
predict(unique_train,test)

Prediction:  2.50



,Domain,Title,Description,Body,Analyst_Rank,Analyst_Average_Score
9,manometcurrent.com,Global Regional Beta Carotene Market Research ...,key players DSM BASF Allied Biotech Chr histor...,Global Regional Beta Carotene Market Research ...,4,0.0


Trained By


,Domain,Title,Description,Body,Analyst_Rank,Analyst_Average_Score
8,news24.com,News24 com Court dismisses attempt Eskom CEO p...,Lawsuit Against Public Participation designed ...,Former Eskom CEO Matshela Moses Koko sought R5...,3,0.33
5,mype.co.za,Male arrested murder elderly female Cofimvaba ...,Crime Stamp Out N1 R101 roads appear court SAP...,South African Police Service Office Provincial...,2,1.33
